In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer, classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
from collections import defaultdict

In [2]:
teamPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/teamData.csv'
playerPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/playerData.csv'
outcomePath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/outcomeData.csv'

In [3]:
teamData = pd.read_csv(teamPath)
playerData = pd.read_csv(playerPath, engine='python')
outcomeData = pd.read_csv(outcomePath)

In [4]:
outcomeData = outcomeData.drop(["Start Time", "Box Score", "Notes"], axis=1)

outcomeData.head()

,Year,Month,Visitor,VisitorPTS,Home,HomePTS,OT?,Attend,Total Points,Winner
0,2005,november,Houston Rockets,79,Detroit Pistons,87,NaN,22076,166,Detroit Pistons
1,2005,november,Sacramento Kings,98,Dallas Mavericks,107,NaN,20041,205,Dallas Mavericks
2,2005,november,Denver Nuggets,78,Los Angeles Lakers,89,NaN,18997,167,Los Angeles Lakers
3,2005,november,Indiana Pacers,109,Cleveland Cavaliers,104,2OT,19730,213,Indiana Pacers
4,2005,november,Milwaukee Bucks,92,Orlando Magic,93,NaN,15138,185,Orlando Magic


In [5]:
def redefineTeamNames(dataframe):
    
    dataframe['Visitor'] = dataframe['Visitor'].replace('Seattle SuperSonics', 'Oklahoma City Thunder', regex=True)
    dataframe['Home'] = dataframe['Home'].replace('Seattle SuperSonics', 'Oklahoma City Thunder', regex=True)
    
    dataframe['Visitor'] = dataframe['Visitor'].replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans', regex=True)
    dataframe['Home'] = dataframe['Home'].replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans', regex=True)

    dataframe['Visitor'] = dataframe['Visitor'].replace('New Orleans Hornets', 'New Orleans Pelicans', regex=True)
    dataframe['Home'] = dataframe['Home'].replace('New Orleans Hornets', 'New Orleans Pelicans', regex=True)
    
    dataframe['Visitor'] = dataframe['Visitor'].replace('Charlotte Bobcats', 'Charlotte Hornets', regex=True)
    dataframe['Home'] = dataframe['Home'].replace('Charlotte Bobcats', 'Charlotte Hornets', regex=True)
    
    dataframe['Visitor'] = dataframe['Visitor'].replace('New Jersey Nets', 'Brooklyn Nets', regex=True)
    dataframe['Home'] = dataframe['Home'].replace('New Jersey Nets', 'Brooklyn Nets', regex=True)

    return dataframe

In [6]:
outcomeData = redefineTeamNames(outcomeData)

In [7]:
def homeWin(dataframe):
    if dataframe['Winner'] == dataframe['Home']:
        return True
    else:
        return False

In [8]:
outcomeData['HomeWin'] = outcomeData.apply(homeWin, axis=1)
encoding = LabelEncoder()
encoding.fit(outcomeData['Home'].values)
outcomeData['Home'] = encoding.transform(outcomeData['Home'].values)
outcomeData['Visitor'] = encoding.transform(outcomeData['Visitor'].values)


In [9]:
# target column (in this case, whether or not the home team will win)
y = outcomeData["HomeWin"]

# columns that will be used to make the prediction
X = outcomeData[["Visitor", "Home"]]

# set up scorer for testing accuracy
scorer = make_scorer(f1_score, pos_label = None, average = "weighted")

In [10]:
# Set up train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=100)

In [11]:
clf = RandomForestClassifier(random_state = 14)
scores = cross_val_score(clf, X_train, y_train, scoring=scorer)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimato

In [12]:
print(np.mean(scores))

0.5840262296053994
